# Segmenting Clustering Assignment
## Requirements
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.
Start by creating a new Notebook for this assignment. Use the Notebook to build the code to scrape the 
following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:





## PREPROCESSING

In [7]:
# importing necessary libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize 



In [8]:
# getting data from internet
# using beautiful soup to parse the HTML/XML codes.
# https://pypi.python.org/pypi/beautifulsoup4
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')



BeautifulSoup  & csv imported.


In [9]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')
#print(soup.prettify()
print('soup ready')

soup ready


In [10]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
    if row != [] and row[1] != "Not assigned":
        # If a cell has a borough but a "Not assigned" neighborhood, then the neighborhood will be the same as the borough.
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
#df.head()

In [11]:
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
#df.head()

In [12]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
#df.head()

In [13]:
df1 = df.reset_index()
#df1.head()

In [14]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 4 columns):
index           180 non-null int64
PostalCode      180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: int64(1), object(3)
memory usage: 5.7+ KB


In [15]:
df1.shape

(180, 4)

More than one neighborhood can exist in one postal code area, M5A is listed twice and has two neighborhoods Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma using groupby, see:

https://pandas-docs.github.io/pandas-docs-travis/user_guide/groupby.html

In [16]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

#df2.head()
       

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180 entries, M1A
 to M9Z

Data columns (total 2 columns):
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(2)
memory usage: 4.2+ KB


In [18]:
df2["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
#df2.head()

In [19]:
df["Borough"] = df["Borough"].str.replace("\n","")
#df.head()

In [20]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
2,M3A\n,North York,Parkwoods
3,M4A\n,North York,Victoria Village
4,M5A\n,Downtown Toronto,Regent Park / Harbourfront
5,M6A\n,North York,Lawrence Manor / Lawrence Heights
6,M7A\n,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [21]:
df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


There are also cells that have an assigned neighbouhoods,like M7A, lets assign their boroughs as their neighbourhood, as follows:

In [22]:
df2 = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [23]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [24]:
df2.shape

(103, 3)

In [25]:
df2.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [26]:
df2 = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df2 = df[~df['PostalCode'].isnull()]  # to filter out bad rows
df2.head(14)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [27]:
df2.shape

(103, 3)

# 2. Using Geocoder package to get the latitude and the longitude coordinates of each neighborhood.

In [80]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim 

In [96]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

#get html from wiki page and create soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'lxml')

#using soup object, iterate the .wikitable to get the data from the HTML page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
    #First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)

#convert list into Pandas DataFrame
canada_df = pd.DataFrame(data = data,columns = columns)
canada_df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [97]:
#Remove Boroughs that are 'Not assigned'
canada_df = canada_df[canada_df['Borough'] != 'Not assigned']
canada_df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [98]:
# More than one neighborhood can exist in one postal code area, combine these into one row with the neighborhoods separated with a comma
canada_df["Neighborhood"] = canada_df.groupby("Postal code")["Neighborhood"].transform(lambda neigh: ', '.join(neigh))

#remove duplicates
canada_df = canada_df.drop_duplicates()

#update index to be postcode if it isn't already
if(canada_df.index.name != 'Postal code'):
    canada_df = canada_df.set_index('Postal code')
    
canada_df.head()

,Borough,Neighborhood
Postal code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Regent Park / Harbourfront
M6A,North York,Lawrence Manor / Lawrence Heights
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [99]:
canada_df.shape

(103, 2)

In [100]:
#Get data lat/long data from csv
lat_long_coord_df = pd.read_csv("http://cocl.us/Geospatial_data")

#rename columns and set the index to be Postcode
lat_long_coord_df.columns = ["Postcode", "Latitude", "Longitude"]
if(lat_long_coord_df.index.name != 'Postcode'):
    lat_long_coord_df = lat_long_coord_df.set_index('Postcode')
    
lat_long_coord_df.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [101]:
canada_df = canada_df.join(lat_long_coord_df)
canada_df.head(11)

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494
M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
M3B,North York,Don Mills,43.745906,-79.352188
M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937
